## Endpoints

In [1]:
MARLOWE_RT_WEBSERVER_URL=http://services.marlowe.run:13780

In [2]:
SIGNING_SERVICE_URL=http://services.marlowe.run:13779

## Addresses and keys

### Party

In [3]:
cat party.testnet.address

addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs

In [4]:
cat party.skey

{
    "type": "PaymentSigningKeyShelley_ed25519",
    "description": "Payment Signing Key",
    "cborHex": "5820bb921c6af621b3336460e6df335d73d4d91012718240bb95dd0301fd9aa5b5e6"
}


### Counterparty

In [5]:
cat counterparty.testnet.address

addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw

## Contract

In [6]:
cat contract.json

{"when":[{"then":{"when":[{"then":{"when":[{"then":"close","case":{"notify_if":true}}],"timeout_continuation":"close","timeout":1685111280000},"case":{"for_choice":{"choice_owner":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"},"choice_name":"Release"},"choose_between":[{"to":1,"from":1}]}}],"timeout_continuation":"close","timeout":1685024580000},"case":{"party":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"},"of_token":{"token_name":"","currency_symbol":""},"into_account":{"address":"addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw"},"deposits":10000000}}],"timeout_continuation":"close","timeout":1684937880000}


In [7]:
json2yaml contract.json

timeout: 1684937880000
timeout_continuation: close
when:
- case:
    deposits: 10000000
    into_account:
      address: addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs
  then:
    timeout: 1685024580000
    timeout_continuation: close
    when:
    - case:
        choose_between:
        - from: 1
          to: 1
        for_choice:
          choice_name: Release
          choice_owner:
            address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs
      then:
        timeout: 1685111280000
        timeout_continuation: close
        when:
        - case:
            notify_if: true
          then: close


## Transaction 1. Create

### Prepare creation request

In [8]:
yaml2json << EOI > request-1.json
version: v1
contract: `cat contract.json`
roles: null
minUTxODeposit: 2000000
metadata: {}
tags: {}
EOI

cat request-1.json

{"contract":{"timeout":1684937880000,"timeout_continuation":"close","when":[{"case":{"deposits":10000000,"into_account":{"address":"addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw"},"of_token":{"currency_symbol":"","token_name":""},"party":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"}},"then":{"timeout":1685024580000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":1,"to":1}],"for_choice":{"choice_name":"Release","choice_owner":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"}}},"then":{"timeout":1685111280000,"timeout_continuation":"close","when":[{"case":{"notify_if":true},"then":"close"}]}}]}}]},"metadata":{},"minUTxODeposit":2000000,"roles":null,"tags":{},"version":"v1"}


### Call endpoint to build creation transaction

In [9]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H 'X-Change-Address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs' \
  -d @request-1.json \
  -o response-1.json \
  -sS

json2yaml response-1.json

links:
  contract: contracts/72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787%231
resource:
  contractId: 72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787#1
  txBody:
    cborHex: 86a40081825820bab33e55c9bdfd179627708120a4fea8b762ab671dfa27cac27085730d90bfe2000182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ad57829a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820a10bfb9106a6faf7ba77592d438bda4bd019cd33aa71fd954f0a9853a5bdd831021a0002d5e50b582067aa14f520459dae714839cd931c1627ac8965c515ab0002a4f59b42cb9145019fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81ffd87a80ffffd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffd87a

### Find contract ID

In [10]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787#1


### Find contract endpoint

In [12]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_URL = http://192.168.0.12:13780/contracts/72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787%231


### Extract unsigned transaction body

In [13]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

cat tx-1.unsigned

{
  "cborHex": "86a40081825820bab33e55c9bdfd179627708120a4fea8b762ab671dfa27cac27085730d90bfe2000182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ad57829a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820a10bfb9106a6faf7ba77592d438bda4bd019cd33aa71fd954f0a9853a5bdd831021a0002d5e50b582067aa14f520459dae714839cd931c1627ac8965c515ab0002a4f59b42cb9145019fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81ffd87a80ffffd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffd87a9f1a00989680ffffd87c9f9fd8799fd87a9fd8799f4752656c65617365d8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffff9fd8799f0101ffffffd87c9f9fd8799fd87b9fd9

### Prepare signing request

In [14]:
yaml2json << EOI > signing-request-1.json
body: $(cat tx-1.unsigned)
paymentKeys: [$(cat party.skey)]
paymentExtendedKeys: []
EOI

cat signing-request-1.json

{"body":{"cborHex":"86a40081825820bab33e55c9bdfd179627708120a4fea8b762ab671dfa27cac27085730d90bfe2000182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ad57829a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820a10bfb9106a6faf7ba77592d438bda4bd019cd33aa71fd954f0a9853a5bdd831021a0002d5e50b582067aa14f520459dae714839cd931c1627ac8965c515ab0002a4f59b42cb9145019fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81ffd87a80ffffd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffd87a9f1a00989680ffffd87c9f9fd8799fd87a9fd8799f4752656c65617365d8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffff9fd8799f0101ffffffd87c9f9fd8799fd87b

### Call endpoint to sign transaction

In [15]:
curl -sS \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -d @signing-request-1.json \
  -o signing-response-1.json \
  $SIGNING_SERVICE_URL/sign
  
json2yaml signing-response-1.json

tx:
  cborHex: 84a40081825820bab33e55c9bdfd179627708120a4fea8b762ab671dfa27cac27085730d90bfe2000182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ad57829a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820a10bfb9106a6faf7ba77592d438bda4bd019cd33aa71fd954f0a9853a5bdd831021a0002d5e50b582067aa14f520459dae714839cd931c1627ac8965c515ab0002a4f59b42cb914501a20081825820ae56efbd10931edd0bcc4c1f929ab9c8d30fe15a58ea4df515aa281d1479a21c5840f188224579981972235874dd65847e79c69a8cd4145262f9c9893600001b9ec84d9df34f498c650fa81b4e53f483595ff6aed2882d513227ff7cb6a52cd04b0e0481d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81ffd87a80ffffd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4

### Extract the text envelope for the signed transaction

In [16]:
jq '.tx' signing-response-1.json > tx-1.signed

### Call the endpoint to submit the transaction

In [17]:
curl -sS \
  -X PUT \
  -H 'Content-Type: application/json' \
  -d @tx-1.signed \
  $CONTRACT_URL

### Poll to see if the transaction has been confirmed

In [18]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-1.json \
  $CONTRACT_URL
  
jq -r '.resource.status' status-1.json

submitted


Wait a little while and then try again.

In [22]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-1.json \
  $CONTRACT_URL
  
jq -r '.resource.status' status-1.json

confirmed


## Transaction 2. Deposit

<font color="red">Note that this step is slightly different from the "create" case.</a>

### Prepare deposit request

In [23]:
yaml2json << EOI > request-2.json
version: v1
inputs:
- input_from_party:
    address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs
  that_deposits: 10000000
  of_token:
    currency_symbol: ''
    token_name: ''
  into_account:
    address: addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw
metadata: {}
tags: {}
EOI

cat request-2.json

{"inputs":[{"input_from_party":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"},"into_account":{"address":"addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":10000000}],"metadata":{},"tags":{},"version":"v1"}


### Call endpoint to build deposit transaction

In [24]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H 'X-Change-Address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs' \
  -d @request-2.json \
  -o response-2.json \
  -sS

json2yaml response-2.json

links:
  transaction: contracts/72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787%231/transactions/bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b
resource:
  contractId: 72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787#1
  transactionId: bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b
  txBody:
    cborHex: 86ab008282582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c7870082582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787010d8182582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c7870012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a33caf3a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b000282005820eedcf09ff9b156c1c19ce17bb5d8b2fb5ccc8e09760320ff6aaadd4cc1b9814310a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ac7d618111a000

### Find transaction endpoint

In [25]:
TRANSACTION_2_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.transaction' response-2.json`"
echo "TRANSACTION_2_URL = $TRANSACTION_2_URL"

TRANSACTION_2_URL = http://192.168.0.12:13780/contracts/72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787%231/transactions/bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b


### Extract unsigned transaction body

In [26]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

cat tx-2.unsigned

{
  "cborHex": "86ab008282582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c7870082582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787010d8182582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c7870012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a33caf3a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b000282005820eedcf09ff9b156c1c19ce17bb5d8b2fb5ccc8e09760320ff6aaadd4cc1b9814310a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ac7d618111a000da211021a000916b6031a01be6418081a01bd47bc0e81581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f8890b58205ee748664f4da5e15abd72fe16695f5f8b35de615e70778a4409fd0a7e7900e49fff82d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799f

### Prepare signing request

In [27]:
yaml2json << EOI > signing-request-2.json
body: $(cat tx-2.unsigned)
paymentKeys: [$(cat party.skey)]
paymentExtendedKeys: []
EOI

cat signing-request-2.json

{"body":{"cborHex":"86ab008282582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c7870082582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787010d8182582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c7870012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a33caf3a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b000282005820eedcf09ff9b156c1c19ce17bb5d8b2fb5ccc8e09760320ff6aaadd4cc1b9814310a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ac7d618111a000da211021a000916b6031a01be6418081a01bd47bc0e81581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f8890b58205ee748664f4da5e15abd72fe16695f5f8b35de615e70778a4409fd0a7e7900e49fff82d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8

### Call endpoint to sign transaction

In [28]:
curl -sS \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -d @signing-request-2.json \
  -o signing-response-2.json \
  $SIGNING_SERVICE_URL/sign
  
json2yaml signing-response-2.json

tx:
  cborHex: 84ab008282582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c7870082582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787010d8182582072a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c7870012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a33caf3a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b000282005820eedcf09ff9b156c1c19ce17bb5d8b2fb5ccc8e09760320ff6aaadd4cc1b9814310a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ac7d618111a000da211021a000916b6031a01be6418081a01bd47bc0e81581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f8890b58205ee748664f4da5e15abd72fe16695f5f8b35de615e70778a4409fd0a7e7900e4a30081825820ae56efbd10931edd0bcc4c1f929ab9c8d30fe15a58ea4df515aa281d1479a21c5840589d1b37af58ff9f814940405c291f866b440afaa723f51cef4078877401b2b92707fe0b9537d6f94debe06f7029d83a01623044a

### Extract the text envelope for the signed transaction

In [29]:
jq '.tx' signing-response-2.json > tx-2.signed

### Call the endpoint to submit the transaction

In [30]:
curl -sS \
  -X PUT \
  -H 'Content-Type: application/json' \
  -d @tx-2.signed \
  $TRANSACTION_2_URL

### Poll to see if the transaction has been confirmed

In [31]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-2.json \
  $TRANSACTION_2_URL
  
jq -r '.resource.status' status-2.json

submitted


Wait a little while and then try again.

In [32]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-2.json \
  $TRANSACTION_2_URL
  
jq -r '.resource.status' status-2.json

confirmed


## Transaction 3. Choice

### Prepare choice request

<font color="red">Note that only this step is different from the "deposit" case. Everything else is the same.</a>

In [33]:
yaml2json << EOI > request-3.json
version: v1
inputs:
- input_that_chooses_num: 1
  for_choice_id:
    choice_name: Release
    choice_owner:
      address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs
metadata: {}
tags: {}
EOI

cat request-3.json

{"inputs":[{"for_choice_id":{"choice_name":"Release","choice_owner":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"}},"input_that_chooses_num":1}],"metadata":{},"tags":{},"version":"v1"}


### Call endpoint to build deposit transaction

In [34]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H 'X-Change-Address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs' \
  -d @request-3.json \
  -o response-3.json \
  -sS

json2yaml response-3.json

links:
  transaction: contracts/72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787%231/transactions/dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd8552
resource:
  contractId: 72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787#1
  transactionId: dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd8552
  txBody:
    cborHex: 86ab0082825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b00825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b010d81825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a2b438ba300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b000282005820b62a06189b3ef1463688be29ed94f12facbf6f5b4384a236d93a9e4115552f7d10a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a26ffd7111a000

### Find transaction endpoint

In [35]:
TRANSACTION_3_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.transaction' response-3.json`"
echo "TRANSACTION_3_URL = $TRANSACTION_3_URL"

TRANSACTION_3_URL = http://192.168.0.12:13780/contracts/72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787%231/transactions/dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd8552


### Extract unsigned transaction body

In [36]:
jq '.resource.txBody' response-3.json > tx-3.unsigned

cat tx-3.unsigned

{
  "cborHex": "86ab0082825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b00825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b010d81825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a2b438ba300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b000282005820b62a06189b3ef1463688be29ed94f12facbf6f5b4384a236d93a9e4115552f7d10a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a26ffd7111a000ccb1c021a00088768031a01bfb6c4081a01bd48290e81581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f8890b58206eee498b97005cdba4b5be663762434f0fbaede0fe1164be70b46809bc555e2e9fff82d8799fd8799f40ffd8799fa2d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480d8799fd8799fd87980d8799fd879

### Prepare signing request

In [37]:
yaml2json << EOI > signing-request-3.json
body: $(cat tx-3.unsigned)
paymentKeys: [$(cat party.skey)]
paymentExtendedKeys: []
EOI

cat signing-request-3.json

{"body":{"cborHex":"86ab0082825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b00825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b010d81825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a2b438ba300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b000282005820b62a06189b3ef1463688be29ed94f12facbf6f5b4384a236d93a9e4115552f7d10a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a26ffd7111a000ccb1c021a00088768031a01bfb6c4081a01bd48290e81581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f8890b58206eee498b97005cdba4b5be663762434f0fbaede0fe1164be70b46809bc555e2e9fff82d8799fd8799f40ffd8799fa2d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480d8799fd8799fd87980d8799f

### Call endpoint to sign transaction

In [38]:
curl -sS \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -d @signing-request-3.json \
  -o signing-response-3.json \
  $SIGNING_SERVICE_URL/sign
  
json2yaml signing-response-3.json

tx:
  cborHex: 84ab0082825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b00825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b010d81825820bf85c5aff0261e78b5cbb945f33a2d252da0540b566946f6c0061ea25039935b0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a2b438ba300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b000282005820b62a06189b3ef1463688be29ed94f12facbf6f5b4384a236d93a9e4115552f7d10a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a26ffd7111a000ccb1c021a00088768031a01bfb6c4081a01bd48290e81581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f8890b58206eee498b97005cdba4b5be663762434f0fbaede0fe1164be70b46809bc555e2ea30081825820ae56efbd10931edd0bcc4c1f929ab9c8d30fe15a58ea4df515aa281d1479a21c5840efa06b31d07396a3f423dd71ebe966fafe610322f22dfab075e69d7b21f7aed9aa1c73bb5393310273c3b368f209d066e9d7ad216

### Extract the text envelope for the signed transaction

In [39]:
jq '.tx' signing-response-3.json > tx-3.signed

### Call the endpoint to submit the transaction

In [40]:
curl -sS \
  -X PUT \
  -H 'Content-Type: application/json' \
  -d @tx-3.signed \
  $TRANSACTION_3_URL

### Poll to see if the transaction has been confirmed

In [41]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-3.json \
  $TRANSACTION_3_URL
  
jq -r '.resource.status' status-3.json

submitted


Wait a little while and then try again.

In [42]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-3.json \
  $TRANSACTION_3_URL
  
jq -r '.resource.status' status-3.json

confirmed


## Transaction 4. Notify

### Prepare notify request

<font color="red">Note that only this step is different from the "deposit" and "choice" cases. Everything else is the same.</a>

In [43]:
yaml2json << EOI > request-4.json
version: v1
inputs:
- input_notify
metadata: {}
tags: {}
EOI

cat request-4.json

{"inputs":["input_notify"],"metadata":{},"tags":{},"version":"v1"}


### Call endpoint to build notify transaction

In [44]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H 'X-Change-Address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs' \
  -d @request-4.json \
  -o response-4.json \
  -sS

json2yaml response-4.json

links:
  transaction: contracts/72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787%231/transactions/cfd7a50fb33773c76aa83cc71eb8e5aaf9c6139a4b0ca54a120870fbc09f81a5
resource:
  contractId: 72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787#1
  transactionId: cfd7a50fb33773c76aa83cc71eb8e5aaf9c6139a4b0ca54a120870fbc09f81a5
  txBody:
    cborHex: 86aa0082825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd855200825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd8552010d81825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd85520012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a233d81a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a001e8480a200581d60c4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81011a0098968010a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a1f3a7c111

### Find transaction endpoint

In [45]:
TRANSACTION_4_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.transaction' response-4.json`"
echo "TRANSACTION_4_URL = $TRANSACTION_4_URL"

TRANSACTION_4_URL = http://192.168.0.12:13780/contracts/72a559b7a7a51f89cd2d09962a566a11f1fc1d5d8bab629ee70a54b91033c787%231/transactions/cfd7a50fb33773c76aa83cc71eb8e5aaf9c6139a4b0ca54a120870fbc09f81a5


### Extract unsigned transaction body

In [46]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

cat tx-4.unsigned

{
  "cborHex": "86aa0082825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd855200825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd8552010d81825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd85520012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a233d81a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a001e8480a200581d60c4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81011a0098968010a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a1f3a7c111a000c090f021a0008060a031a01c10970081a01bd48570b58205b0d07cb65380645561b7f5e8f386383d4318c629605a5328eddc3304bca951d9fff81d8799fd8799f40ffd8799fa2d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480d8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be

### Prepare signing request

In [47]:
yaml2json << EOI > signing-request-4.json
body: $(cat tx-4.unsigned)
paymentKeys: [$(cat party.skey)]
paymentExtendedKeys: []
EOI

cat signing-request-4.json

{"body":{"cborHex":"86aa0082825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd855200825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd8552010d81825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd85520012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a233d81a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a001e8480a200581d60c4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81011a0098968010a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a1f3a7c111a000c090f021a0008060a031a01c10970081a01bd48570b58205b0d07cb65380645561b7f5e8f386383d4318c629605a5328eddc3304bca951d9fff81d8799fd8799f40ffd8799fa2d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480d8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd44

### Call endpoint to sign transaction

In [48]:
curl -sS \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -d @signing-request-4.json \
  -o signing-response-4.json \
  $SIGNING_SERVICE_URL/sign
  
json2yaml signing-response-4.json

tx:
  cborHex: 84aa0082825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd855200825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd8552010d81825820dac5bb4208db9f43c0a35394837d6ec96cfe42f6eaabfb973e00d5f22cbd85520012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a233d81a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a001e8480a200581d60c4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81011a0098968010a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0a1f3a7c111a000c090f021a0008060a031a01c10970081a01bd48570b58205b0d07cb65380645561b7f5e8f386383d4318c629605a5328eddc3304bca951da30081825820ae56efbd10931edd0bcc4c1f929ab9c8d30fe15a58ea4df515aa281d1479a21c58406ba52bd9caad7e7dce86efb09411876d21b8677f6ffae35976ba452c50a6b6e64f98741a91ba8ddbbf7a31009e8fc4944f8dadf1d77cc7faf6ad580dc4dfe8050481d8799fd8799f40ffd8799fa2d8799fd87

### Extract the text envelope for the signed transaction

In [49]:
jq '.tx' signing-response-4.json > tx-4.signed

### Call the endpoint to submit the transaction

In [50]:
curl -sS \
  -X PUT \
  -H 'Content-Type: application/json' \
  -d @tx-4.signed \
  $TRANSACTION_4_URL

### Poll to see if the transaction has been confirmed

In [51]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-4.json \
  $TRANSACTION_4_URL
  
jq -r '.resource.status' status-4.json

submitted


Wait a little while and then try again.

In [52]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-4.json \
  $TRANSACTION_4_URL
  
jq -r '.resource.status' status-4.json

confirmed
